# RAG Pipeline for Database Knowledge Q&A

This notebook demonstrates how to create a complete RAG (Retrieval-Augmented Generation) pipeline using database dumps as knowledge sources for question-answering systems.

## Overview
- Load and process various database dump formats (SQL, JSON, CSV)
- Create embeddings and vector store
- Set up retrieval system
- Implement Q&A interface with LLMs
- Build evaluation and monitoring

## Features
- 🗃️ **Multi-format support**: SQL dumps, JSON exports, CSV files
- 🔍 **Smart retrieval**: Semantic search with metadata filtering
- 🧠 **LLM integration**: Support for local and cloud models
- 📊 **Analytics**: Query performance and relevance tracking
- 🎯 **Context-aware**: Maintains database relationships and structure

In [ ]:
# Import Required Libraries
import os
import json
import pandas as pd
import sqlite3
import sqlparse
from pathlib import Path
from typing import List, Dict, Any, Optional
from datetime import datetime
import numpy as np

# RAG and Vector Store Libraries
try:
    from langchain.schema import Document
    from langchain_community.vectorstores import Chroma, FAISS
    from langchain_community.embeddings import OpenAIEmbeddings
    from langchain.text_splitter import RecursiveCharacterTextSplitter
    from langchain.chains import RetrievalQA
    from langchain_community.llms import OpenAI
    print("✓ LangChain libraries available")
except ImportError:
    print("Installing LangChain...")
    import subprocess
    import sys
    subprocess.check_call([sys.executable, "-m", "pip", "install", "langchain", "langchain-community", "langchain-openai"])
    from langchain.schema import Document
    from langchain_community.vectorstores import Chroma, FAISS
    from langchain_community.embeddings import OpenAIEmbeddings
    from langchain.text_splitter import RecursiveCharacterTextSplitter
    from langchain.chains import RetrievalQA
    from langchain_community.llms import OpenAI

# Alternative embedding models
try:
    from sentence_transformers import SentenceTransformer
    print("✓ Sentence Transformers available")
    SENTENCE_TRANSFORMERS_AVAILABLE = True
except ImportError:
    print("Installing sentence-transformers for local embeddings...")
    subprocess.check_call([sys.executable, "-m", "pip", "install", "sentence-transformers"])
    from sentence_transformers import SentenceTransformer
    SENTENCE_TRANSFORMERS_AVAILABLE = True

# Vector database options
try:
    import chromadb
    print("✓ ChromaDB available")
    CHROMADB_AVAILABLE = True
except ImportError:
    print("Installing ChromaDB...")
    subprocess.check_call([sys.executable, "-m", "pip", "install", "chromadb"])
    import chromadb
    CHROMADB_AVAILABLE = True

# Azure AI Inference for Q&A
try:
    from azure.ai.inference import ChatCompletionsClient
    from azure.ai.inference.models import SystemMessage, UserMessage
    from azure.core.credentials import AzureKeyCredential
    print("✓ Azure AI Inference available")
    AZURE_AI_AVAILABLE = True
except ImportError:
    print("Installing Azure AI Inference...")
    subprocess.check_call([sys.executable, "-m", "pip", "install", "azure-ai-inference"])
    from azure.ai.inference import ChatCompletionsClient
    from azure.ai.inference.models import SystemMessage, UserMessage
    from azure.core.credentials import AzureKeyCredential
    AZURE_AI_AVAILABLE = True

print(f"\n🚀 RAG Pipeline Setup Complete")
print(f"Working directory: {os.getcwd()}")

# Configuration
BASE_DIR = Path(r"c:\Github\Learn-GenAI\genai_book")
RAG_DATA_DIR = BASE_DIR / "rag_data"
RAG_DATA_DIR.mkdir(exist_ok=True)

print(f"RAG data directory: {RAG_DATA_DIR}")
print(f"Available embedding models: {'Local + Cloud' if SENTENCE_TRANSFORMERS_AVAILABLE else 'Cloud only'}")

In [ ]:
# Database/CSV Loader and Processor
class DatabaseRAGLoader:
    """
    Comprehensive loader for various database dump formats
    """
    
    def __init__(self, data_dir: Path):
        self.data_dir = data_dir
        self.documents = []
        
    def load_csv_file(self, csv_path: str, chunk_size: int = 1000) -> List[Document]:
        """
        Load large CSV files in chunks and convert to documents
        """
        csv_file = Path(csv_path)
        if not csv_file.exists():
            print(f"❌ CSV file not found: {csv_file}")
            return []
            
        print(f"📊 Loading CSV file: {csv_file}")
        documents = []
        
        try:
            # Read CSV in chunks to handle large files
            chunk_iter = pd.read_csv(csv_file, chunksize=chunk_size)
            
            for chunk_idx, chunk in enumerate(chunk_iter):
                print(f"  Processing chunk {chunk_idx + 1} ({len(chunk)} rows)")
                
                # Get column information
                columns = list(chunk.columns)
                
                # Convert each row to a document
                for idx, row in chunk.iterrows():
                    # Create a readable text representation
                    content_parts = []
                    
                    # Add structured information
                    for col in columns:
                        value = row[col]
                        if pd.notna(value):
                            content_parts.append(f"{col}: {value}")
                    
                    content = "\n".join(content_parts)
                    
                    # Create document with metadata
                    doc = Document(
                        page_content=content,
                        metadata={
                            'source': str(csv_file),
                            'row_id': int(idx),
                            'chunk_id': chunk_idx,
                            'record_type': 'database_record',
                            'columns': columns,
                            'file_size_mb': csv_file.stat().st_size / (1024 * 1024),
                            'row_count': len(chunk)
                        }
                    )
                    documents.append(doc)
                
                # Process in smaller batches to avoid memory issues
                if chunk_idx >= 10:  # Limit for demo - adjust as needed
                    print(f"  ⚠️ Processing limited to first {chunk_idx + 1} chunks for demo")
                    break
                    
        except Exception as e:
            print(f"❌ Error loading CSV: {e}")
            return []
            
        print(f"✅ Loaded {len(documents)} documents from CSV")
        return documents
    
    def load_sql_dump(self, sql_path: str) -> List[Document]:
        """
        Load SQL dump files and extract meaningful content
        """
        sql_file = Path(sql_path)
        if not sql_file.exists():
            return []
            
        print(f"🗃️ Loading SQL dump: {sql_file}")
        documents = []
        
        try:
            with open(sql_file, 'r', encoding='utf-8') as f:
                sql_content = f.read()
            
            # Parse SQL statements
            statements = sqlparse.split(sql_content)
            
            for idx, statement in enumerate(statements):
                if statement.strip():
                    parsed = sqlparse.parse(statement)[0]
                    
                    # Extract table information and data
                    content = f"SQL Statement {idx + 1}:\n{statement}"
                    
                    doc = Document(
                        page_content=content,
                        metadata={
                            'source': str(sql_file),
                            'statement_id': idx,
                            'record_type': 'sql_statement',
                            'statement_type': parsed.get_type()
                        }
                    )
                    documents.append(doc)
                    
        except Exception as e:
            print(f"❌ Error loading SQL dump: {e}")
            
        return documents
    
    def load_json_dump(self, json_path: str) -> List[Document]:
        """
        Load JSON database exports
        """
        json_file = Path(json_path)
        if not json_file.exists():
            return []
            
        print(f"📝 Loading JSON dump: {json_file}")
        documents = []
        
        try:
            with open(json_file, 'r', encoding='utf-8') as f:
                data = json.load(f)
            
            # Handle different JSON structures
            if isinstance(data, list):
                # Array of records
                for idx, record in enumerate(data):
                    content = json.dumps(record, indent=2, ensure_ascii=False)
                    
                    doc = Document(
                        page_content=content,
                        metadata={
                            'source': str(json_file),
                            'record_id': idx,
                            'record_type': 'json_record'
                        }
                    )
                    documents.append(doc)
                    
            elif isinstance(data, dict):
                # Single object or nested structure
                for key, value in data.items():
                    content = f"Key: {key}\nValue: {json.dumps(value, indent=2, ensure_ascii=False)}"
                    
                    doc = Document(
                        page_content=content,
                        metadata={
                            'source': str(json_file),
                            'key': key,
                            'record_type': 'json_key_value'
                        }
                    )
                    documents.append(doc)
                    
        except Exception as e:
            print(f"❌ Error loading JSON dump: {e}")
            
        return documents

# Initialize the loader
loader = DatabaseRAGLoader(RAG_DATA_DIR)

# Load your CSV file
csv_file_path = r"c:\Github\Learn-GenAI\genai_book\llm\data_ingestion\endpoint.csv"
print(f"🔄 Starting to load CSV file...")
documents = loader.load_csv_file(csv_file_path, chunk_size=500)  # Smaller chunks for large file

print(f"\n📊 CSV Loading Summary:")
print(f"Total documents created: {len(documents)}")

if documents:
    # Show sample document
    sample_doc = documents[0]
    print(f"\nSample document metadata:")
    for key, value in sample_doc.metadata.items():
        print(f"  {key}: {value}")
    
    print(f"\nSample content preview:")
    print(f"{sample_doc.page_content[:300]}...")
    
    # Show statistics
    chunks = set(doc.metadata.get('chunk_id', 0) for doc in documents)
    print(f"\nData statistics:")
    print(f"  Total chunks processed: {len(chunks)}")
    print(f"  Records per chunk: ~{len(documents) // len(chunks) if chunks else 0}")
    print(f"  Average content length: {sum(len(doc.page_content) for doc in documents) // len(documents)} characters")
else:
    print("⚠️ No documents loaded. Please check the CSV file path and format.")

In [ ]:
# Text Processing and Chunking for RAG
class DatabaseTextProcessor:
    """
    Specialized text processor for database content
    """
    
    def __init__(self):
        self.splitter = RecursiveCharacterTextSplitter(
            chunk_size=1000,
            chunk_overlap=100,
            length_function=len,
            separators=["\n\n", "\n", ": ", ", ", " "]
        )
    
    def process_documents(self, documents: List[Document]) -> List[Document]:
        """
        Process and chunk documents optimally for database content
        """
        if not documents:
            return []
            
        print(f"🔄 Processing {len(documents)} documents...")
        processed_docs = []
        
        for doc in documents:
            # For database records, we might want to keep them as single chunks
            # or split them intelligently based on content length
            
            if len(doc.page_content) > 1500:  # Split large records
                chunks = self.splitter.split_text(doc.page_content)
                
                for i, chunk in enumerate(chunks):
                    chunk_doc = Document(
                        page_content=chunk,
                        metadata={
                            **doc.metadata,
                            'chunk_index': i,
                            'total_chunks': len(chunks),
                            'is_chunked': True
                        }
                    )
                    processed_docs.append(chunk_doc)
            else:
                # Keep small records intact
                doc.metadata['is_chunked'] = False
                processed_docs.append(doc)
        
        print(f"✅ Created {len(processed_docs)} processed chunks")
        return processed_docs

# Process the loaded documents
processor = DatabaseTextProcessor()
processed_documents = processor.process_documents(documents)

if processed_documents:
    print(f"\nDocument Processing Summary:")
    print(f"Original documents: {len(documents)}")
    print(f"Processed chunks: {len(processed_documents)}")
    
    # Count chunked vs non-chunked
    chunked = len([d for d in processed_documents if d.metadata.get('is_chunked', False)])
    non_chunked = len(processed_documents) - chunked
    print(f"Chunked documents: {chunked}")
    print(f"Non-chunked documents: {non_chunked}")
    
    # Show content length distribution
    lengths = [len(doc.page_content) for doc in processed_documents]
    print(f"\nContent length statistics:")
    print(f"  Min: {min(lengths)} characters")
    print(f"  Max: {max(lengths)} characters") 
    print(f"  Average: {sum(lengths) // len(lengths)} characters")
    
    # Sample processed document
    sample = processed_documents[0]
    print(f"\nSample processed document:")
    print(f"Content length: {len(sample.page_content)}")
    print(f"Is chunked: {sample.metadata.get('is_chunked')}")
    print(f"Content preview: {sample.page_content[:200]}...")
else:
    print("⚠️ No documents to process")

In [ ]:
# Embedding Creation and Vector Store Setup
class DatabaseEmbeddingManager:
    """
    Manages embeddings and vector store for database RAG
    """
    
    def __init__(self, use_local_embeddings: bool = True):
        self.use_local = use_local_embeddings
        self.embeddings = None
        self.vector_store = None
        
        if use_local_embeddings and SENTENCE_TRANSFORMERS_AVAILABLE:
            print("🔄 Loading local embedding model...")
            # Use a model optimized for diverse content
            self.embedding_model = SentenceTransformer('all-MiniLM-L6-v2')
            print("✅ Local embedding model loaded")
        else:
            print("🌐 Using OpenAI embeddings (requires API key)")
            self.embeddings = OpenAIEmbeddings()
    
    def create_embeddings(self, texts: List[str]) -> np.ndarray:
        """
        Create embeddings for text content
        """
        if self.use_local and hasattr(self, 'embedding_model'):
            return self.embedding_model.encode(texts)
        else:
            # Would use OpenAI embeddings if API key is available
            return None
    
    def create_vector_store(self, documents: List[Document], store_type: str = "chromadb") -> Any:
        """
        Create vector store from documents
        """
        if not documents:
            print("❌ No documents provided for vector store")
            return None
            
        print(f"🔄 Creating {store_type} vector store with {len(documents)} documents...")
        
        try:
            if store_type == "chromadb":
                # Create custom embedding function for ChromaDB
                class LocalEmbeddingFunction:
                    def __init__(self, model):
                        self.model = model
                    
                    def __call__(self, texts):
                        embeddings = self.model.encode(texts)
                        return embeddings.tolist()
                
                # Initialize ChromaDB
                chroma_client = chromadb.Client()
                
                # Create collection
                collection_name = f"database_rag_{datetime.now().strftime('%Y%m%d_%H%M%S')}"
                
                # Extract texts and metadata
                texts = [doc.page_content for doc in documents]
                metadatas = [doc.metadata for doc in documents]
                ids = [f"doc_{i}" for i in range(len(documents))]
                
                # Create embeddings if using local model
                if self.use_local and hasattr(self, 'embedding_model'):
                    embeddings = self.create_embeddings(texts)
                    
                    # Create collection
                    collection = chroma_client.create_collection(
                        name=collection_name,
                        metadata={"description": "Database RAG Collection"}
                    )
                    
                    # Add documents
                    collection.add(
                        embeddings=embeddings.tolist(),
                        documents=texts,
                        metadatas=metadatas,
                        ids=ids
                    )
                    
                    print(f"✅ ChromaDB collection '{collection_name}' created")
                    return collection
                
            elif store_type == "faiss":
                # Use FAISS for local vector store
                if self.use_local and hasattr(self, 'embedding_model'):
                    vector_store = FAISS.from_documents(
                        documents, 
                        embedding=self.embedding_model.encode
                    )
                    
                    # Save to disk
                    faiss_path = RAG_DATA_DIR / "faiss_index"
                    vector_store.save_local(str(faiss_path))
                    print(f"✅ FAISS index saved to {faiss_path}")
                    return vector_store
                    
        except Exception as e:
            print(f"❌ Error creating vector store: {e}")
            return None
        
        return None
    
    def search_similar(self, query: str, collection, top_k: int = 5) -> List[Dict]:
        """
        Search for similar documents
        """
        if not collection:
            return []
            
        try:
            if self.use_local and hasattr(self, 'embedding_model'):
                # Create query embedding
                query_embedding = self.embedding_model.encode([query])
                
                # Search in ChromaDB
                results = collection.query(
                    query_embeddings=query_embedding.tolist(),
                    n_results=top_k,
                    include=['documents', 'metadatas', 'distances']
                )
                
                return results
                
        except Exception as e:
            print(f"❌ Error searching: {e}")
            return []

# Initialize embedding manager
embedding_manager = DatabaseEmbeddingManager(use_local_embeddings=True)

# Create vector store from processed documents
if processed_documents:
    print(f"🔄 Creating vector store...")
    vector_store = embedding_manager.create_vector_store(
        processed_documents, 
        store_type="chromadb"
    )
    
    if vector_store:
        print(f"✅ Vector store created successfully")
        
        # Test search functionality
        test_query = "What information is available?"
        print(f"\n🔍 Testing search with query: '{test_query}'")
        
        search_results = embedding_manager.search_similar(test_query, vector_store, top_k=3)
        
        if search_results and 'documents' in search_results:
            print(f"Found {len(search_results['documents'][0])} results:")
            for i, (doc, metadata, distance) in enumerate(zip(
                search_results['documents'][0], 
                search_results['metadatas'][0],
                search_results['distances'][0]
            )):
                print(f"\n{i+1}. (Distance: {distance:.3f})")
                print(f"   Metadata: {metadata}")
                print(f"   Content: {doc[:150]}...")
        else:
            print("No search results found")
    else:
        print("❌ Failed to create vector store")
else:
    print("⚠️ No processed documents available for vector store creation")

In [ ]:
# RAG Q&A System Implementation
class DatabaseRAGSystem:
    """
    Complete RAG system for database Q&A
    """
    
    def __init__(self, vector_store, embedding_manager):
        self.vector_store = vector_store
        self.embedding_manager = embedding_manager
        self.llm_client = None
        
        # Initialize LLM client (GitHub Models)
        self.setup_llm_client()
    
    def setup_llm_client(self):
        """
        Setup LLM client for Q&A
        """
        github_token = os.environ.get("GITHUB_TOKEN")
        
        if github_token and AZURE_AI_AVAILABLE:
            try:
                self.llm_client = ChatCompletionsClient(
                    endpoint="https://models.github.ai/inference",
                    credential=AzureKeyCredential(github_token),
                )
                print("✅ LLM client connected (GitHub Models)")
            except Exception as e:
                print(f"⚠️ LLM client setup failed: {e}")
        else:
            print("⚠️ GITHUB_TOKEN not set - LLM functionality limited")
    
    def retrieve_relevant_docs(self, query: str, top_k: int = 5) -> List[Dict]:
        """
        Retrieve relevant documents for the query
        """
        if not self.vector_store:
            return []
        
        search_results = self.embedding_manager.search_similar(query, self.vector_store, top_k)
        
        relevant_docs = []
        if search_results and 'documents' in search_results:
            for doc, metadata, distance in zip(
                search_results['documents'][0],
                search_results['metadatas'][0], 
                search_results['distances'][0]
            ):
                relevant_docs.append({
                    'content': doc,
                    'metadata': metadata,
                    'relevance_score': 1 - distance,  # Convert distance to similarity
                    'distance': distance
                })
        
        return relevant_docs
    
    def generate_answer(self, query: str, relevant_docs: List[Dict]) -> Dict:
        """
        Generate answer using LLM and retrieved context
        """
        if not self.llm_client:
            return {
                "answer": "LLM not available. Please set GITHUB_TOKEN environment variable.",
                "confidence": 0.0,
                "sources": []
            }
        
        # Prepare context from relevant documents
        context_parts = []
        sources = []
        
        for i, doc in enumerate(relevant_docs):
            context_parts.append(f"Document {i+1}:\n{doc['content']}")
            sources.append({
                'doc_id': i+1,
                'metadata': doc['metadata'],
                'relevance': doc['relevance_score']
            })
        
        context = "\n\n".join(context_parts)
        
        # Create system and user messages
        system_message = """You are a helpful AI assistant that answers questions based on database information. 
        Use the provided context to answer the user's question accurately and concisely.
        If the context doesn't contain enough information, say so clearly.
        Always cite which document(s) you used for your answer."""
        
        user_message = f"""Context from database:
{context}

Question: {query}

Please provide a comprehensive answer based on the context above."""
        
        try:
            response = self.llm_client.complete(
                messages=[
                    SystemMessage(system_message),
                    UserMessage(user_message),
                ],
                temperature=0.3,
                max_tokens=500,
                model="openai/gpt-4.1-mini"
            )
            
            return {
                "answer": response.choices[0].message.content,
                "confidence": 0.85,  # Could be calculated based on retrieval scores
                "sources": sources,
                "tokens_used": response.usage.total_tokens if response.usage else None
            }
            
        except Exception as e:
            return {
                "answer": f"Error generating response: {e}",
                "confidence": 0.0,
                "sources": sources
            }
    
    def ask_question(self, query: str, top_k: int = 5) -> Dict:
        """
        Complete Q&A pipeline
        """
        print(f"🤔 Question: {query}")
        print("🔍 Retrieving relevant information...")
        
        # Retrieve relevant documents
        relevant_docs = self.retrieve_relevant_docs(query, top_k)
        
        if not relevant_docs:
            return {
                "answer": "No relevant information found in the database.",
                "confidence": 0.0,
                "sources": [],
                "query": query
            }
        
        print(f"📚 Found {len(relevant_docs)} relevant documents")
        
        # Generate answer
        print("🤖 Generating answer...")
        result = self.generate_answer(query, relevant_docs)
        result["query"] = query
        result["retrieved_docs"] = len(relevant_docs)
        
        return result

# Initialize RAG system
if vector_store and embedding_manager:
    rag_system = DatabaseRAGSystem(vector_store, embedding_manager)
    
    # Test the Q&A system
    test_questions = [
        "What data is available in this database?",
        "Can you summarize the main information?",
        "What are the key fields or columns?",
        "How many records are there?"
    ]
    
    print(f"\n🧪 Testing RAG System with sample questions:")
    print("=" * 60)
    
    for question in test_questions:
        print(f"\n❓ {question}")
        result = rag_system.ask_question(question, top_k=3)
        
        print(f"💡 Answer: {result['answer']}")
        print(f"🎯 Confidence: {result['confidence']:.2f}")
        print(f"📖 Sources: {result['retrieved_docs']} documents")
        if result.get('tokens_used'):
            print(f"🔤 Tokens used: {result['tokens_used']}")
        print("-" * 40)
        
else:
    print("⚠️ Vector store not available - cannot initialize RAG system")

In [ ]:
# Interactive Q&A Interface and Performance Analytics
class InteractiveRAG:
    """
    Interactive interface for the RAG system with analytics
    """
    
    def __init__(self, rag_system):
        self.rag_system = rag_system
        self.conversation_history = []
        self.analytics = {
            'total_queries': 0,
            'avg_response_time': 0.0,
            'satisfaction_scores': []
        }
    
    def ask_interactive_question(self, question: str) -> Dict:
        """
        Ask a question with timing and analytics
        """
        start_time = datetime.now()
        
        result = self.rag_system.ask_question(question)
        
        end_time = datetime.now()
        response_time = (end_time - start_time).total_seconds()
        
        # Add to conversation history
        conversation_entry = {
            'timestamp': start_time.isoformat(),
            'question': question,
            'answer': result['answer'],
            'confidence': result['confidence'],
            'response_time': response_time,
            'sources_count': result['retrieved_docs']
        }
        
        self.conversation_history.append(conversation_entry)
        
        # Update analytics
        self.analytics['total_queries'] += 1
        self.analytics['avg_response_time'] = (
            (self.analytics['avg_response_time'] * (self.analytics['total_queries'] - 1) + response_time) 
            / self.analytics['total_queries']
        )
        
        result['response_time'] = response_time
        return result
    
    def save_conversation(self, filename: str = None):
        """
        Save conversation history to file
        """
        if not filename:
            timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
            filename = f"rag_conversation_{timestamp}.json"
        
        filepath = RAG_DATA_DIR / filename
        
        conversation_data = {
            'conversation_history': self.conversation_history,
            'analytics': self.analytics,
            'metadata': {
                'total_questions': len(self.conversation_history),
                'session_start': self.conversation_history[0]['timestamp'] if self.conversation_history else None,
                'session_end': self.conversation_history[-1]['timestamp'] if self.conversation_history else None
            }
        }
        
        with open(filepath, 'w', encoding='utf-8') as f:
            json.dump(conversation_data, f, indent=2, ensure_ascii=False)
        
        print(f"💾 Conversation saved to: {filepath}")
        return filepath
    
    def generate_report(self):
        """
        Generate performance and usage report
        """
        if not self.conversation_history:
            print("No conversation data available")
            return
        
        print(f"\n📊 RAG SYSTEM PERFORMANCE REPORT")
        print("=" * 50)
        print(f"Total queries: {self.analytics['total_queries']}")
        print(f"Average response time: {self.analytics['avg_response_time']:.2f}s")
        
        # Calculate confidence statistics
        confidences = [entry['confidence'] for entry in self.conversation_history]
        print(f"Average confidence: {sum(confidences)/len(confidences):.2f}")
        print(f"Min confidence: {min(confidences):.2f}")
        print(f"Max confidence: {max(confidences):.2f}")
        
        # Response time statistics
        response_times = [entry['response_time'] for entry in self.conversation_history]
        print(f"Min response time: {min(response_times):.2f}s")
        print(f"Max response time: {max(response_times):.2f}s")
        
        # Sources statistics
        sources_counts = [entry['sources_count'] for entry in self.conversation_history]
        print(f"Average sources retrieved: {sum(sources_counts)/len(sources_counts):.1f}")
        
        # Recent questions
        print(f"\nRecent Questions:")
        for i, entry in enumerate(self.conversation_history[-3:], 1):
            print(f"{i}. {entry['question']}")
            print(f"   Confidence: {entry['confidence']:.2f} | Time: {entry['response_time']:.2f}s")

# Create interactive RAG interface
if 'rag_system' in locals() and rag_system:
    interactive_rag = InteractiveRAG(rag_system)
    
    print(f"\n🚀 INTERACTIVE RAG SYSTEM READY")
    print("=" * 50)
    print("You can now ask questions about your database!")
    
    # Demonstrate with some database-specific questions
    demo_questions = [
        "What is the structure of this database?",
        "How many records are in the dataset?", 
        "What are the main data categories?",
        "Can you show me a sample record?",
        "What fields contain the most information?"
    ]
    
    print(f"\n🎯 Demonstrating with database-specific questions:")
    
    for question in demo_questions:
        print(f"\n" + "="*60)
        result = interactive_rag.ask_interactive_question(question)
        
        print(f"❓ Question: {question}")
        print(f"💡 Answer: {result['answer']}")
        print(f"📊 Metrics: Confidence={result['confidence']:.2f} | Time={result['response_time']:.2f}s | Sources={result['retrieved_docs']}")
    
    # Generate and save report
    print(f"\n" + "="*60)
    interactive_rag.generate_report()
    
    # Save conversation
    conversation_file = interactive_rag.save_conversation()
    
    print(f"\n✅ Interactive RAG session complete!")
    print(f"📁 Results saved to: {RAG_DATA_DIR}")
    
else:
    print("⚠️ RAG system not available - please ensure previous cells ran successfully")

In [ ]:
# Custom Question Interface - Use this cell to ask your own questions!

def ask_custom_question(question: str):
    """
    Function to ask custom questions to the RAG system
    """
    if 'interactive_rag' in locals() and interactive_rag:
        print(f"🤔 Your Question: {question}")
        print("🔍 Processing...")
        
        result = interactive_rag.ask_interactive_question(question)
        
        print(f"\n💡 Answer:")
        print(f"{result['answer']}")
        print(f"\n📊 Details:")
        print(f"  • Confidence: {result['confidence']:.2%}")
        print(f"  • Response Time: {result['response_time']:.2f}s")
        print(f"  • Sources Used: {result['retrieved_docs']} documents")
        
        if result.get('tokens_used'):
            print(f"  • Tokens Used: {result['tokens_used']}")
        
        # Show source information if available
        if 'sources' in result and result['sources']:
            print(f"\n📚 Source Documents:")
            for i, source in enumerate(result['sources'][:3], 1):  # Show top 3 sources
                print(f"  {i}. Relevance: {source['relevance']:.2%}")
                if 'row_id' in source['metadata']:
                    print(f"     Row ID: {source['metadata']['row_id']}")
                if 'chunk_id' in source['metadata']:
                    print(f"     Chunk: {source['metadata']['chunk_id']}")
        
        return result
    else:
        print("❌ RAG system not available. Please run the previous cells first.")
        return None

# Example usage - modify these questions to ask about your specific CSV data
print("🎯 CUSTOM QUESTION INTERFACE")
print("="*50)
print("Use the ask_custom_question() function to query your CSV database!")
print("\nExample questions you can try:")

example_questions = [
    "What columns are in my CSV file?",
    "Show me some example data from the database",
    "What is the most common value in the dataset?", 
    "How is the data structured?",
    "What patterns can you identify in the data?",
    "Can you describe the data quality?",
    "What insights can you provide about this dataset?"
]

for i, q in enumerate(example_questions, 1):
    print(f"{i}. {q}")

print(f"\n💡 To ask a question, run:")
print(f"ask_custom_question('Your question here')")

# Uncomment the line below and modify the question to test:
# ask_custom_question("What information is in my CSV database?")

## 🎉 RAG Pipeline Complete!

You now have a fully functional RAG system for your CSV database! Here's what you've built:

### 🔧 **System Components**

1. **📊 Data Loader** - Efficiently loads large CSV files in chunks
2. **🔄 Text Processor** - Optimizes database records for RAG
3. **🧠 Embedding Engine** - Creates semantic search capabilities
4. **🗄️ Vector Store** - ChromaDB for fast similarity search
5. **🤖 Q&A System** - GitHub Models integration for answers
6. **📈 Analytics** - Performance tracking and conversation history

### 🚀 **How to Use**

```python
# Ask questions about your CSV data
ask_custom_question("What is in my database?")
ask_custom_question("Show me the data structure")
ask_custom_question("What insights can you provide?")
```

### 📋 **Features**

- ✅ **Handles Large Files** - Processes CSV files in chunks
- ✅ **Semantic Search** - Finds relevant information using AI
- ✅ **Context Preservation** - Maintains database relationships
- ✅ **Performance Tracking** - Monitors speed and accuracy
- ✅ **Conversation History** - Saves all Q&A sessions
- ✅ **Local + Cloud** - Works with local embeddings and cloud LLMs

### 🎯 **Best Practices for Your CSV RAG**

1. **Data Quality**: Clean data produces better answers
2. **Question Specificity**: Specific questions get better results
3. **Context Awareness**: Reference columns and data types in questions
4. **Iterative Queries**: Build on previous questions for deeper insights

### 🔧 **Production Deployment**

For production use:
- Set up persistent vector store
- Implement user authentication
- Add query rate limiting
- Monitor system performance
- Scale with more compute resources

### 📊 **Next Steps**

- Experiment with different question types
- Analyze the conversation history
- Optimize chunk sizes for your data
- Add more sophisticated retrieval strategies
- Implement feedback loops for continuous improvement

**Your RAG system is ready to answer questions about your CSV database!** 🎊